In [1]:
import math
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pathlib import Path
import pandas as pd

# In[3]:

MODEL_PATH = Path.cwd()/'Models'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
class CustomDataset(Dataset):
    def __init__(self, df, transform=None, target_transform=None):
        self.df = df
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        x = self.df.drop(labels='AGE', axis=1).iloc[idx]
        x = torch.from_numpy(x.values)
        #returning y as a scalar might be a problem
        y = self.df.iloc[idx].AGE
        if self.transform:
            x = self.transform(x)
        if self.target_transform:
            y = self.target_transform(y)
        return x, y


# In[5]:


class NN(nn.Module):
    def __init__(self, layers=[1000,500,250], ps=0.35, in_features=20, y_range=(20, 90)):
        super(NN, self).__init__()
        self.y_range = y_range
        self.layers = layers
        self.ps = ps
        layers = [in_features] + layers
        layers = list(zip(layers, layers[1:]))
        
        l = []
        for layer in layers:
            l.append(nn.Linear(*layer))
            #TODO: play with negative slope koef. of LeakyReLU
            l.append(nn.LeakyReLU())
            l.append(nn.Dropout(ps))
        l.append(nn.Linear(layers[-1][1], 1))

        self.arch = nn.Sequential(*l)
        
    def forward(self, x):
        x = self.arch(x)
        x = (self.y_range[1]-self.y_range[0]) * torch.sigmoid(x) + self.y_range[0]
        return x
    
    def __repr__(self):
        return "Linear -> LeakyReLU -> Dropout\nlayers: {}\nps: {}\n".format(self.layers, self.ps)

##### COPY FROM AGECLOCK MODULE^^^^

In [3]:
import ipywidgets as widgets
from ipywidgets import Button, Layout
from ipywidgets import GridspecLayout, Button, Layout, Dropdown, BoundedFloatText

In [4]:
import torch
from pathlib import Path

In [5]:
def predict(biomarkers):
    path = Path.cwd()/'Models'/'adm_state_dict'
    lm = NN()
    lm.load_state_dict(torch.load(path))
    lm.eval()
    biomarkers = torch.Tensor(biomarkers)
    return(lm(biomarkers))

In [12]:
Path.cwd()/'Models'

15

In [ ]:
('second index of the model path list var', [x for x in (Path.cwd()/'Models').iterdir() if x.is_file()][-2])

In [ ]:
('hand written code', Path.cwd()/'Models'/'adm_state_dict', (Path.cwd()/'Models'/'adm_state_dict').exists())

In [6]:
from ipywidgets import *
x = widgets.Dropdown(options=['a', 'b'], description="Load an example")
y = widgets.Dropdown(options=["Male", "Female"], description="Sex:")

def change_x(*args):
    y.index = None
    y.value = "Female"
    
    for i in range(len(MALE40)+1):
        grid[i+1, 2].value = MALE40[i]
        
x.observe(change_x, 'value')
HBox([x,y])

In [7]:
def submit_clicked(b):
    markers = [grid[i+1, 2].value for i in range(len(biomarkers))]
    markers.insert(0, int(y.value=='Male'))
    with output:
        print(f"You are {round(predict(markers).data.item())} years old.")
        
#MALe = 1 
#FEmale = 0
#Like www.aging.ai Western Europe except for:
#Albumine, Creatine urine isntead of blood
#Protein total/10 

MALE40 = [
    49.21,
    5.28,
    7.3,
    7.04,
    7,
    143,
    120,
    16.5,
    17.2,
    1.35,
    1.6,
    4.83,
    2.48,
    4.7,
    46.7,
    35.4,
    86.5,
    177,
    5.41
]

In [8]:
biomarkers = [
    "Albumin_mgl",
    "Glucose_mmolL",
    "Urea_mmolL",
    "Cholesterol_mmolL",
    "Protein_gdL",
    "Sodium_mmolL",
    "Creatinine_mgdl",
    "Hemoglobin_gdl",
    "Bilirubin_umolL",
    "Triglyceride_mmolL",
    "HDL_mmolL",
    "LDL_mmolL",
    "Calcium_mmolL",
    "Potassium_mmolL",
    "Hematocrit_%",
    "MCHC_gdl",
    "MCV_fL",
    "Platelet_TuL",
    "RBC_MuL"]

In [9]:
biomarkers = [(i+1, biomarkers[i].split('_')[0], biomarkers[i].split('_')[1]) for i in range(len(biomarkers))]

In [10]:
from ipywidgets import GridspecLayout

grid = widgets.GridspecLayout(len(biomarkers)+2, 5, height='800px', width='auto')
h = '28px'
for x in biomarkers:
        grid[x[0], 1] = Button(description=x[1], tooltip=x[1], button_style="info", disabled=True, layout=Layout(height=h, width='auto'))
        grid[x[0], 3] = Button(description=x[2], tooltip=x[2], button_style="info", disabled=True, layout=Layout(height=h, width='auto'))
        
        grid[x[0], 2] = BoundedFloatText(
        min=0,
        max=1000,
        layout=Layout(width='auto', height=h),
        value=None,
        step=0.001,
        #description=' ',
        disabled=False
    )
        
grid[0,1] = Button(description="Blood Marker*", tooltip="Units and Sample Metric***", button_style='primary', disabled=True, layout=Layout(height='auto', width='auto'))
grid[0,2] = Button(description="Your Value", tooltip="Your Value", button_style="primary", disabled=True, layout=Layout(height='auto', width='auto'))
grid[0,3] = Button(description="Units and Sample Metric***", tooltip="Units and Sample Metric***", button_style="primary", disabled=True, layout=Layout(height='auto', width='auto'))
        
grid

GridspecLayout(children=(Button(button_style='info', description='Albumin', disabled=True, layout=Layout(grid_…

In [11]:
button = widgets.Button(description="Submit")
button.on_click(submit_clicked)
output = widgets.Output()
display(button, output)

Button(description='Submit', style=ButtonStyle())

Output()